In [9]:
# 导入必要的库
from datasets import load_dataset
import pandas as pd
from typing import Dict, List
import os
os.environ["HF_ENDPOINT"]="https://hf-mirror.com"

# 加载 HuggingFace 数据集并处理
def process_dataset(dataset_name: str, data_source: str, key: str = "proof", include_verify: bool = False) -> pd.DataFrame:
    # 加载数据集
    dataset = load_dataset(dataset_name)
    
    # 假设使用 train 分割，你可以根据需要修改
    data = dataset["train"]
    # data=dataset["validation"]
    
    # 初始化结果列表
    results = []
    
    # 遍历数据集
    for item in data:
        # 获取 proof 字段
        proof = item[key]
        
        # 获取 formal_statement 并按要求分割
        formal_statement_full = proof
        # formal_statement_full = proof.replace("set_option maxHeartbeats 0","set_option maxHeartbeats 500000")
        if ":=" in formal_statement_full:
            formal_statement = formal_statement_full.split(":=")[0] + ":= by"
            formal_statement="import Mathlib\nimport Aesop\nset_option maxHeartbeats 500000\nopen BigOperators\nopen Real\nopen Nat\nopen Topology\n"+formal_statement
        else:
            continue
        if len(formal_statement)> 1000 :
            continue
            
        # 根据 include_verify 参数决定是否包含验证文本
        verify_text = ""
        if include_verify:
            verify_text = "You can also verify your proof by using <verify> your_code </verify> and it will return error messages and 'sorry' information if there are issues. "
            
        # 构造 prompt
        prompt = [{
            "content": f"""Finish the given Lean4 proof. \
You must conduct reasoning inside <think> and </think> first every time you get new information. \
After reasoning, if you find you need to use a theorem, you can search mathlib4 by <search> math_statement </search> and it will return relevant theorems between <information> and </information>. \
You can search as many times as your want. \
{verify_text}\
If you find no further theorems needed, you can directly provide the completed proof inside <answer> and </answer>. Question: {formal_statement}\n""",
            "role": "user"
        }]
        
        # 构造 reward_model
        reward_model = {
            "ground_truth": {
                "formal_statement": formal_statement
            }
        }
        
        # 构造 extra_info
        extra_info = {k: v for k, v in item.items()}
        
        # 添加到结果
        results.append({
            "data_source": data_source,
            "prompt": prompt,
            "ability": "math",
            "reward_model": reward_model,
            "extra_info": extra_info
        })
    
    # 转换为 DataFrame
    df = pd.DataFrame(results)
    return df

# 示例用法
dataset_name = "pkuAI4M/LeanWorkbook"  # 替换为实际的 HuggingFace 数据集名称
data_source = "lean_workbook_full"    # 由用户指定
df = process_dataset(dataset_name, data_source, key="formal_statement")

# 显示前几行


Using the latest cached version of the dataset since pkuAI4M/LeanWorkbook couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/pkuAI4M___lean_workbook/default/0.0.0/b7b59dd09fb6e771290ed616a7037bc62a16842a (last modified on Tue Mar  4 12:57:51 2025).


In [10]:
len(df)

140172

In [11]:
pd.set_option('display.max_colwidth', None)
print(df.head())

          data_source  \
0  lean_workbook_full   
1  lean_workbook_full   
2  lean_workbook_full   
3  lean_workbook_full   
4  lean_workbook_full   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     prompt  \
0            [{'content': 'Finish this proof in Lean4 code:

import Mathlib
import Aesop
set_option maxHeartbeats 500000
open BigOperators
open Real
open Nat
open Topology
theorem lean_workbook_0 (a b c : ℝ) (ha : 0 < a) (hb : 0 < b) (hc : 0 < c) : (b + c) / Real.sqrt (a ^ 2 + 8 * b * c) + (c + a) / Real.

In [13]:
def split_and_save_dataset(df: pd.DataFrame, output_dir: str, test_size: int = 16):
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 随机打乱数据
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # 拆分为测试集和训练集
    test_df = df_shuffled.iloc[:test_size]
    train_df = df_shuffled.iloc[test_size:]
    
    # 保存为 parquet 文件
    test_path = os.path.join(output_dir, "test.parquet")
    train_path = os.path.join(output_dir, "train.parquet")
    
    test_df.to_parquet(test_path, index=False)
    train_df.to_parquet(train_path, index=False)
    
    print(f"Test set saved to: {test_path} ({len(test_df)} rows)")
    print(f"Train set saved to: {train_path} ({len(train_df)} rows)")

# 示例用法

output_dir = "/AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/leanwkbk_full"  # 替换为指定的输出路径

# 处理数据集

# 拆分并保存
split_and_save_dataset(df, output_dir, test_size=16)

Test set saved to: /AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/leanwkbk_full/test.parquet (16 rows)
Train set saved to: /AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/leanwkbk_full/train.parquet (140156 rows)


In [6]:
def save_dataset(df: pd.DataFrame, output_dir: str, test_size: int = 16):
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 随机打乱数据
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # 拆分为测试集和训练集
    test_df = df_shuffled
    
    # 保存为 parquet 文件
    test_path = os.path.join(output_dir, "test.parquet")

    
    test_df.to_parquet(test_path, index=False)

    
    print(f"Test set saved to: {test_path} ({len(test_df)} rows)")


# 示例用法

output_dir = "/AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/minif2f"  # 替换为指定的输出路径

# 处理数据集

# 拆分并保存
save_dataset(df, output_dir, test_size=16)

Test set saved to: /AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/minif2f/test.parquet (244 rows)
